In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/playground-series-s3e20/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df=df.drop(columns=("ID_LAT_LON_YEAR_WEEK"),axis=1)

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df.iloc[1:,:])
dfl=pd.DataFrame(imputer.fit_transform(df),
             columns = imputer.get_feature_names_out())

In [ ]:
dfl.isna().sum()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(df.iloc[:,-2],df["emission"],alpha=0.3,marker="*")
plt.xlabel("cloud_solar")
plt.ylabel("emission")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
X=np.asanyarray(dfl.drop(['emission'],axis=1))
y=np.asanyarray(dfl.emission)

pipe=Pipeline([("std",StandardScaler()),("reg",RandomForestRegressor(n_estimators=1, random_state=0))])
pipe.fit(X, y)

In [ ]:
pipe.score(X,y)

In [ ]:
pred=pipe.predict(X)
pred

In [ ]:
df_test=pd.read_csv("/kaggle/input/playground-series-s3e20/test.csv")
df_test=df_test.drop(columns=("ID_LAT_LON_YEAR_WEEK"),axis=1)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(df_test.iloc[1:,:])
df_test=pd.DataFrame(imputer.fit_transform(df_test),
             columns = imputer.get_feature_names_out())

In [ ]:
predict_test=pipe.predict(df_test.values)
predict_test                        

In [ ]:
sub_file=pd.read_csv("/kaggle/input/playground-series-s3e20/sample_submission.csv")

In [ ]:
sub_file = pd.DataFrame({'ID_LAT_LON_YEAR_WEEK': sub_file.ID_LAT_LON_YEAR_WEEK, 'emission': predict_test})
sub_file.head()

In [ ]:
submission=sub_file.to_csv('Submission.csv')